In [1]:
import pandas as pd
import os

In [5]:
interactions = pd.read_csv('mind/mind.inter', sep='\t')

# user_id:token	
# item_id:token	
# timestamp:float	
# num_repeat:float

In [6]:
users = set(interactions['user_id:token'])
items = set(interactions['item_id:token'])

res_users = len(users)
res_items = len(items)
n_inters = len(interactions)
sparsity = 1 - (n_inters / (res_users * res_items))
rat_user = n_inters / res_users
rat_item = n_inters / res_items

print(f"\nusers:{res_users}\nitems:{res_items}")
print(f"inters:{n_inters}\nsparsity:{sparsity}")
print(f"rat/user:{rat_user}\nrat_item:{rat_item}")

print('Users:', len(users))
print('Items:', len(items))


users:23679
items:4414
inters:1048575
sparsity:0.9899676237184807
rat/user:44.282908906626126
rat_item:237.55663797009515
Users: 23679
Items: 4414


In [9]:
def stats_core(interactions, k=5):

    user_counts = interactions['user_id:token'].value_counts()
    item_counts = interactions['item_id:token'].value_counts()

    # Filter out users and items that appear less than 5 times
    user_k = user_counts[user_counts >= 5].index
    item_k = item_counts[item_counts >= 5].index

    # Filter the DataFrame based on valid users and items
    interactions_k = interactions[(interactions['user_id:token'].isin(user_k)) & (interactions['item_id:token'].isin(item_k))]

    users = set(interactions_k['user_id:token'])
    items = set(interactions_k['item_id:token'])

    res_users = len(users)
    res_items = len(items)
    n_inters = len(interactions_k)
    sparsity = 1 - (n_inters / (res_users * res_items))
    rat_user = n_inters / res_users
    rat_item = n_inters / res_items

    print(f"\nusers:{res_users}\nitems:{res_items}")
    print(f"inters:{n_inters}\nsparsity:{sparsity}")
    print(f"rat/user:{rat_user}\nrat_item:{rat_item}")

    print('Users:', len(users))
    print('Items:', len(items))
    return interactions_k




In [10]:
interactions_k = stats_core(interactions=interactions, k=5)




users:69878
items:10196
inters:9998816
sparsity:0.9859661030476576
rat/user:143.0896133260826
rat_item:980.6606512357787
Users: 69878
Items: 10196


In [11]:
interactions_k_filterd = interactions_k.drop(columns=['timestamp:float'])

In [12]:
interactions_k_filterd.to_csv('ml-10m_core5/ml-10m_core5.inter', sep='\t', index=False)

In [2]:
import random

def sample_users(interactions=None, n_users=5000, seed=42):

    users = list(set(interactions['user_id:token']))

    random.seed(seed)
    random_users = random.sample(users, n_users)

    filtered_interactions = interactions[interactions['user_id:token'].isin(random_users)]
    
    res_users = len(set(filtered_interactions['user_id:token']))
    res_items = len(set(filtered_interactions['item_id:token']))
    n_inters = len(filtered_interactions)
    sparsity = 1 - (n_inters / (res_users * res_items))
    rat_user = n_inters / res_users
    rat_item = n_inters / res_items

    print(f"\nusers:{res_users}\nitems:{res_items}")
    print(f"inters:{n_inters}\nsparsity:{sparsity}")
    print(f"rat/user:{rat_user}\nrat_item:{rat_item}")

    return filtered_interactions


def sample_users_items(interactions=None, n_users=5000, n_items=5000, seed=42):

    users = list(set(interactions['user_id:token']))
    items = list(set(interactions['item_id:token']))

    random.seed(seed)
    random_users = random.sample(users, n_users)
    random_items = random.sample(items, n_items)

    filtered_interactions = interactions[interactions['user_id:token'].isin(random_users) & interactions['item_id:token'].isin(random_items)]

    res_users = len(set(filtered_interactions['user_id:token']))
    res_items = len(set(filtered_interactions['item_id:token']))
    n_inters = len(filtered_interactions)
    sparsity = 1 - (n_inters / (res_users * res_items))
    rat_user = n_inters / res_users
    rat_item = n_inters / res_items

    print(f"\nusers:{res_users}\nitems:{res_items}")
    print(f"inters:{n_inters}\nsparsity:{sparsity}")
    print(f"rat/user:{rat_user}\nrat_item:{rat_item}")
    filtered_interactions.to_csv('ml-10m_40U10I/ml-10m_40U10I.inter', sep='\t', index=False)
    #return filtered_interactions



In [3]:
#sample_users(interactions=interactions_k_filterd, n_users=5000, seed=42)
#sample_users_items(interactions=interactions_k_filterd, n_users=15000, n_items=15000, seed=42)
#sample_users_items(interactions=interactions_k_filterd, n_users=20000, n_items=30000, seed=42)
#sample_users_items(interactions=interactions_k_filterd, n_users=20000, n_items=40000, seed=42)
interactions_k_filterd = pd.read_csv('ml-10m_core5/ml-10m_core5.inter', sep='\t')
sample_users_items(interactions=interactions_k_filterd, n_users=40000, n_items=10000, seed=42)
print("Done")


users:40000
items:9999
inters:5534016
sparsity:0.9861635763576357
rat/user:138.3504
rat_item:553.4569456945694
Done


In [4]:
# Dal file di inter prendere item presenti, recuperare il nome delle entità corrispondenti dal file .link. Poi andare nel file .kg e filtrare solo le righe che presentano le entità recuperate in precedenza. Salvare il file filtrato con un nome significativo.

#FILTRARE IN BASE ALLA PRIMA COLONNA
#FORSE ANCHE IN BASE ALLA TERZA COLONNA

interactions = pd.read_csv('ml-10m_40U10I/ml-10m_40U10I.inter', sep='\t')
links=pd.read_csv('ml-10m_40U10I/ml-10m_40U10I.link', sep='\t')

#from links save only rows where item_id:token is in interactions
interactions_links = links[links['item_id:token'].isin(set(interactions['item_id:token']))]
print('Interactions:', len(interactions))


entities = interactions_links['entity_id:token']
print('Entities:', len(entities))
kg = pd.read_csv('ml-10m_40U10I/ml-10m_40U10I.kg', sep='\t')


kg_filtered = kg[kg['head_id:token'].isin(entities)]

print('KG:', len(kg_filtered))
os.remove('ml-10m_40U10I/ml-10m_40U10I.kg')
kg_filtered.to_csv('ml-10m_40U10I/ml-10m_40U10I.kg', sep='\t', index=False)


Interactions: 5534016
Entities: 9937
KG: 520645


In [3]:
def dataSetInfo(path:str):
    print("NOME DEL DATASET: ",path)
    print("INFO SULLE INTERAZIONI")
    interactions = pd.read_csv(path+"/"+path+".inter", sep='\t')
    n_users = interactions['user_id:token'].nunique()
    n_items = interactions['item_id:token'].nunique()
    n_interactions = len(interactions)
    print("Numero utenti=",n_users)
    print("Numero items=",n_items)
    print("Numero interazioni=",n_interactions)
    print("Sparsity=",1-(n_interactions/(n_users*n_items)))
    print("Avg interazioni per utente=",n_interactions/n_users)
    print("INFO SUL KNWOLEDGE GRAPH")
    kg=pd.read_csv(path+"/"+path+".kg", sep='\t')
    n_entities_head=kg['head_id:token'].nunique()
    n_entities_tail=kg['tail_id:token'].nunique()
    n_relations=kg['relation_id:token'].nunique()
    n_triples=len(kg)
    print("Numero entità head=",n_entities_head)
    print("Numero entità tail=",n_entities_tail)
    print("Numero relazioni=",n_relations)
    print("Numero triple=",n_triples)
    print("Nomi relazioni=",kg['relation_id:token'].unique())

In [4]:
dataSetInfo("LFM-1b_artist")
print("\n\n\n\n\n")
dataSetInfo("LFM-1b_artist_core5")
print("\n\n\n\n\n")
dataSetInfo("LFM-1b_artist_20U50I")
print("\n\n\n\n\n")
dataSetInfo("LFM-1b_artist_20U50I_75strat")
print("\n\n\n\n\n")
dataSetInfo("LFM-1b_artist_20U50I_50strat")
print("\n\n\n\n\n")
dataSetInfo("LFM-1b_artist_20U50I_25strat")
print("\n\n\n\n\n")

NOME DEL DATASET:  LFM-1b_artist
INFO SULLE INTERAZIONI
Numero utenti= 120322
Numero items= 3123496
Numero interazioni= 65133026
Sparsity= 0.9998266933373666
Avg interazioni per utente= 541.3226675088513
INFO SUL KNWOLEDGE GRAPH
Numero entità head= 823213
Numero entità tail= 353607
Numero relazioni= 8
Numero triple= 2114049
Nomi relazioni= ['music.recording.artist' 'music.recording.releases'
 'music.recording.producer' 'music.recording.engineer'
 'music.recording.featured_artists' 'music.featured_artist.recordings'
 'music.release.artist' 'music.artist.release']






NOME DEL DATASET:  LFM-1b_artist_core5
INFO SULLE INTERAZIONI
Numero utenti= 120175
Numero items= 585095
Numero interazioni= 61534450
Sparsity= 0.9991248594539152
Avg interazioni per utente= 512.0403578115248
INFO SUL KNWOLEDGE GRAPH
Numero entità head= 823213
Numero entità tail= 353607
Numero relazioni= 8
Numero triple= 2114049
Nomi relazioni= ['music.recording.artist' 'music.recording.releases'
 'music.recording.produce